# VarLMM.jl

`VarLMM.jl` is a Julia package for modeling within-subject variability of a longitudinal measurement. 

In addition to mean levels, it can be important to model factors influencing within-subject variability of longitudinal outcomes. We utilize a modified linear mixed effects model where ---. How much methods?

## Installation

This package requires Julia v1.0 or later, which can be obtained from https://julialang.org/downloads/ or by building Julia from the sources in the https://github.com/JuliaLang/julia repository.

The package has not yet been registered and must be installed using the repository location. Start Julia and use the ] key to switch to the package manager REPL

(v1.1) pkg> add TODO: ADD URL

Use the backspace key to return to the Julia REPL.

In [1]:
using VarLMM, CSV

┌ Info: Precompiling VarLMM [2ff19380-1883-49fc-9d10-450face6b90c]
└ @ Base loading.jl:1273


## Example Usage

!!! Make sample data -- read in, etc etc
The example dataset is contained in `vlmm_example.csv`. Here we have a simulated example where systolic blood pressure (sbp) is a function of other covariates.

In [2]:
df = CSV.read("varlmm_testexdata.csv")

,id,sbp,age,gender,bmi,meds
,Int64,Float64,Float64,Int64,Float64,Float64
1,1,135.557,1.0,1,26.0424,0.0
2,1,136.036,1.0,1,26.3548,0.0
3,1,131.133,1.0,1,24.8406,0.0
4,1,131.038,1.0,1,24.0226,0.0
5,1,132.81,1.0,1,25.3946,0.0
6,1,134.346,1.0,1,25.668,0.0
7,1,130.74,1.0,1,24.0091,0.0
8,1,132.704,1.0,1,25.3698,0.0
9,1,136.047,1.0,1,26.0414,0.0


First we will create a VarLmmModel object from the dataframe.

The `VarLmmModel()` function takes the following arguments: 

- `meanformula`: the formula for the mean fixed effects β (variables in X matrix)
- `reformula`: the formula for the mean random effects γ (variables in Z matrix)
- `wsvarformula`: the formula  for the within-subject variance fixed effects τ (variables in W matrix). 
- `idvar`: the id variable for groupings. 
- `df`: the dataframe holding all of the data for the model. 

For documentation of the ordinalgwas function, type `?ordinalgwas` in Julia REPL.

We will model sbp as a function of age, gender, and bmi. The following commands fit the following model:

$sbp_{ij} = \beta_0 + \beta_1 age_{ij} + \beta_2 gender_{ij} + \beta_3 bmi_{ij} + \gamma_{i0} + \gamma_{i1}age_{ij} + \epsilon_{ij}$

$\epsilon_{ij}$ is distributed with mean 0 variance $\sigma^2_{\epsilon_{ij}}$

$\sigma^2_{\epsilon_{ij}} = exp(\tau_0 + \tau_1 age_{ij} + \tau_2 gender_{ij} + \tau_3 bmi_{ij} + \ell_{\gamma \omega}^T \gamma_i + \omega_i)$

In [3]:
vlmm = VarLmmModel(@formula(sbp ~ 1 + age + gender + bmi), 
    @formula(sbp ~ 1 + bmi), 
    @formula(sbp ~ 1 + age + gender + bmi),
    :id, df);

The `vlmm` object has the appropriate data. We can use the `fit!()` function to fit the model.

## Fitting

The `fit()` function fits with the following procedure.

1. Intialize parameters through least squares $(\beta, \tau, L_\gamma)$ for initial starting points.
2. Re-Estimate $(\tau, L_\gamma)$ through the unweighted objective function (method of moments).
3. Re-estimate $\beta$ with weighted least squares using $(\tau, L_\gamma)$ estimates. 
4. For 1:weightedruns
    - 4a. Update working covariance matrix $V_i^{(0)}$ with current estimates of $(\tau, \Sigma_\gamma)$.
    - 4b. Re-estimate $(\tau, \Sigma_\gamma)$ using weighted estimating equations (weighted objective function).
5. Use final estimates and conduct inference.

The `fit()` function takes the following arguments:
* `m::VarLmmModel` the model to fit.
* `solver` by default this is Ipopt.IpoptSolver(print_level=5, watchdog_shortened_iter_trigger=3)
* `weightedruns` by default this is 1.

In [4]:
fit!(vlmm)


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.10, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:       28

Total number of variables............................:        7
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equ

───────────────────────────────────────────────────────
      Estimate  Std. Error  Wald Statistic  Pr(>|Wald|)
───────────────────────────────────────────────────────
β1  80.0671      2.4892            1034.63       <1e-99
β2  12.34        1.22895            100.82       <1e-22
β3  -5.05001     2.05073              6.06       0.0138
β4   0.973764    0.0777087          157.02       <1e-35
τ1  -1.11188     2.66354              0.17       0.6764
τ2   2.20452     0.163782           181.17       <1e-40
τ3   0.0981204   0.163442             0.36       0.5483
τ4  -0.0751532   0.0987367            0.58       0.4466
───────────────────────────────────────────────────────

By default, the `fit()` function fits with the `IPOPT` solver. This can be changed easily by specifying a different solver.

In [5]:
fit!(vlmm, NLopt.NLoptSolver(algorithm = :LD_MMA, maxeval = 4000))

───────────────────────────────────────────────────────
      Estimate  Std. Error  Wald Statistic  Pr(>|Wald|)
───────────────────────────────────────────────────────
β1  80.0229      2.49892           1025.47       <1e-99
β2  12.3636      1.21749            103.12       <1e-23
β3  -5.15518     2.06098              6.26       0.0124
β4   0.975822    0.0751213          168.74       <1e-37
τ1   5.65773     1.436               15.52       <1e-4 
τ2   1.58116     0.0937715          284.32       <1e-63
τ3   0.0989285   0.149426             0.44       0.5079
τ4  -0.268424    0.0543891           24.36       <1e-6 
───────────────────────────────────────────────────────

In [6]:
fit!(vlmm, KNITRO.KnitroSolver(outlev=3))


Knitro 12.1.1 STUDENT LICENSE (problem size limit = 300)

Knitro 12.1.1 STUDENT LICENSE (problem size limit = 300)

            Student License
       (NOT FOR COMMERCIAL USE)
         Artelys Knitro 12.1.1

Knitro presolve eliminated 0 variables and 0 constraints.

outlev:                  3
The problem is identified as unconstrained.
Knitro changing algorithm from AUTO to 1.
Knitro changing bar_initpt from AUTO to 3.
Knitro changing bar_murule from AUTO to 4.
Knitro changing bar_penaltycons from AUTO to 0.
Knitro changing bar_penaltyrule from AUTO to 2.
Knitro changing bar_switchrule from AUTO to 0.
Knitro changing linesearch from AUTO to 2.
Knitro changing linsolver from AUTO to 2.

Problem Characteristics                                 (   Presolved)
-----------------------
Objective goal:  Minimize
Objective type:  general
Number of variables:                                  7 (           7)
    bounded below only:                               0 (           0)
    bounded abov

───────────────────────────────────────────────────────
      Estimate  Std. Error  Wald Statistic  Pr(>|Wald|)
───────────────────────────────────────────────────────
β1  80.0662      2.48936           1034.48       <1e-99
β2  12.3413      1.22909            100.82       <1e-22
β3  -5.04885     2.05085              6.06       0.0138
β4   0.973667    0.0777584          156.79       <1e-35
τ1  -1.13438     2.68433              0.18       0.6726
τ2   2.20465     0.164025           180.66       <1e-40
τ3   0.0982829   0.163787             0.36       0.5485
τ4  -0.0743064   0.0994895            0.56       0.4551
───────────────────────────────────────────────────────

There is also an option to fit the model with only the weighted objective function. In cases where the contribution of the within-subject variance to the overall variance is very different than the contribution of the random effects $\gamma_i$, it can be more stable. The function `fitweightedonly!()` can be used to do this. 

## Weighted Only Estimation Procedure

1. Intialize parameters through least squares $(\beta, \tau, \Sigma)$ for initial starting points.
3. Re-estimate $\beta$ with weighted least squares using $(\tau, \Sigma_\gamma)$ estimates. 
4. For 1:weightedruns
    - 3a. Update $V_i^{(0)}$ with current estimates of $(\tau, \Sigma_\gamma)$.
    - 3b. Re-estimate $(\tau, \Sigma_\gamma)$ using weighted estimating equations (new objective function).
5. Use final estimates and conduct inference.

The `fitweightedonly!()` function takes the following arguments:
* `m::VarLmmModel` the model to fit.
* `solver` by default this is Ipopt.IpoptSolver(print_level=5, watchdog_shortened_iter_trigger=3)
* `weightedruns` by default this is 2.

In [12]:
fitweightedonly!(vlmm; weightedruns=2)

This is Ipopt version 3.12.10, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:       28

Total number of variables............................:        7
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0 

───────────────────────────────────────────────────────
      Estimate  Std. Error  Wald Statistic  Pr(>|Wald|)
───────────────────────────────────────────────────────
β1  80.2541      3.39953            557.31       <1e-99
β2  11.8486      1.42404             69.23       <1e-16
β3  -5.72403     2.46205              5.41       0.0201
β4   1.01764     0.0791199          165.43       <1e-37
τ1  -1.48301     0.78074              3.61       0.0575
τ2   2.12437     0.057047          1386.74       <1e-99
τ3   0.140745    0.0988474            2.03       0.1545
τ4  -0.0510737   0.0315572            2.62       0.1056
───────────────────────────────────────────────────────

## Tips

- If the `fit!()` function is giving errors, a different solver may remedy the issue. By default, `VarLMM` is using the IPOPT solver, but it can take any solver that supports [MathProgBase.jl](https://github.com/JuliaOpt/MathProgBase.jl). In our experience, [Knitro.jl](https://github.com/JuliaOpt/KNITRO.jl) works the best, but it has only a commercial or academic trial period. 

- The `fit()` function can also work poorly if the variance contribution from the random effects heavily outweighs the contribution from the within-subject variance. Standardizing continuous variables can help this immensely.

In [10]:
vlmm2b = VarLmmModel(@formula(sbp ~ 1 + age + gender + bmi + meds), #adding meds as covariate changes this completely
    @formula(sbp ~ 1 + bmi), 
    @formula(sbp ~ 1 + age + gender + bmi),
    :id, df);
fit!(vlmm2b)

This is Ipopt version 3.12.10, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:       28

Total number of variables............................:        7
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0 

InterruptException: InterruptException:

In [11]:
fitweightedonly!(vlmm2b)

This is Ipopt version 3.12.10, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:       28

Total number of variables............................:        7
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0 

──────────────────────────────────────────────────────
     Estimate  Std. Error  Wald Statistic  Pr(>|Wald|)
──────────────────────────────────────────────────────
β1  79.768      3.52848            511.07       <1e-99
β2  11.905      1.43209             69.11       <1e-16
β3  -5.66903    2.46208              5.30       0.0213
β4   1.01802    0.0791042          165.62       <1e-37
β5   1.55162    3.00003              0.27       0.6050
τ1  -0.17849   10.0975               0.00       0.9859
τ2   1.85945    1.90481              0.95       0.3290
τ3   0.256223   0.206115             1.55       0.2138
τ4  -0.074872   0.166559             0.20       0.6531
──────────────────────────────────────────────────────

In [43]:
using Random, LinearAlgebra, Distributions, DataFrames
@info "generate data"
Random.seed!(123)
# dimensions
m  = 500 # number of individuals
ns = rand(9:11, m) # numbers of observations per individual
p  = 5    # number of fixed effects, including intercept
q  = 2    # number of random effects, including intercept
l  = 4    # number of WS variance covariates, including intercept
obsvec = Vector{VarLmmObs{Float64}}(undef, m)
# true parameter values
βtrue = [80.0; 15.0; -2.5; 0.9; 0.75] #intercept, agegroup, #gender, #bmi, #meds
τtrue = [-1.0; 1.0; -1.5; 0.10] #intercept, agegroup, meds, bmi
Σγ    = Matrix(Diagonal([2.0; 0.05]))
δγω   = [0.1; 0.1; rand(q - 2) ./ 10]
σω    = [0.5]
Σγω   = [Σγ δγω; δγω' σω]
Lγω   = cholesky(Symmetric(Σγω), check = false).L
Lγ    = Lγω[1:q, 1:q]
lγω   = Lγω[q + 1, 1:q]
lω    = Lγω[q + 1, q + 1]
# generate data
γω = Vector{Float64}(undef, q + 1)
z  = similar(γω) # hold vector of iid std normal
df = DataFrame(id = String[], sbp = Float64[], age=Float64[], 
    gender=Int64[], bmi = Float64[], meds = Float64[])
for i in 1:m
    # first column intercept, remaining entries iid std normal
    X = Matrix{Float64}(undef, ns[i], p)
    X[:, 1] .= 1
    agegroup = Distributions.rand(1:3) #age
    gender = Distributions.rand(Bernoulli(0.5)) #gender
    meds = Distributions.rand(Bernoulli(0.2)) #meds
    @views fill!(X[:, 2], agegroup)
    @views fill!(X[:, 3], Int(gender))
    @views Distributions.rand!(Normal(21, 1.2), X[:, 4]) #bmi
    @views fill!(X[:, 5], meds)
    # first column intercept, remaining entries iid std normal
    Z = Matrix{Float64}(undef, ns[i], q)
    Z[:, 1] .= 1
    @views copyto!(Z[:, 2], X[:, 4]) #bmi 
    # first column intercept, remaining entries iid std normal
    W = Matrix{Float64}(undef, ns[i], l)
    W[:, 1] .= 1
    @views copyto!(W[:, 2], X[:, 2]) #agegroup
    @views copyto!(W[:, 3], X[:, 5]) #meds
    @views copyto!(W[:, 4], X[:, 4]) #bmi
    # generate random effects: γω = Lγω * z
    mul!(γω, Lγω, Distributions.rand!(Normal(), z))
    # generate y
    μy = X * βtrue + Z * γω[1:q]
    @views vy = exp.(W * τtrue .+ dot(γω[1:q], lγω) .+ γω[end])
    y = rand(MvNormal(μy, Diagonal(vy)))
        if i == 1
        @show vy
        @show μy
        @show y
    end
    id = fill(string(i), ns[i])
    tempdf = DataFrame([id y X[:, 2:p]])
    rename!(tempdf, [:id, :sbp, :age, :gender, :bmi, :meds])
    # form a VarLmmObs instance
    append!(df, tempdf)
end
rename!(df, [:id, :sbp, :age, :gender, :bmi, :meds])


vlmm = VarLmmModel(@formula(sbp ~ 1 + age + gender + bmi + meds), 
    @formula(sbp ~ 1 + bmi), 
    @formula(sbp ~ 1 + age + meds + bmi),
    :id, df);

ReVars = Vector{Float64}(undef, m)
WsVars = Vector{Float64}(undef, m)
RevarWsVarRatio = Vector{Float64}(undef, m)

for i in eachindex(vlmm.data)
    ob = vlmm.data[i]
    ReVars[i] = tr(transpose(ob.Zt) * Lγ * transpose(Lγ) * ob.Zt)
    WsVars[i] = sum(exp.(transpose(ob.Wt) * τtrue))
    RevarWsVarRatio[i] = ReVars[i] / WsVars[i]
end
mean(RevarWsVarRatio), minimum(RevarWsVarRatio), maximum(RevarWsVarRatio)

┌ Info: generate data
└ @ Main In[43]:2


vy = [128.8819466572765, 182.06883264845632, 152.90527245081307, 154.22755818598904, 152.33883209571766, 142.78874790412846, 159.32782104780054, 146.20944645937703, 174.28219600409523]
μy = [146.07929014393636, 149.8909003031373, 147.9649913447289, 148.05998777605822, 147.9240452075447, 147.20978806009222, 148.41892801471388, 147.47097141419638, 149.4086788090492]
y = [133.78549455536088, 142.22063958020644, 136.7121150364212, 139.6338726270973, 159.25456872549836, 135.19505280425884, 143.96111857221942, 148.83791831042015, 165.87522358081912]


(2.1609342179996585, 0.39114044915123364, 13.23876283255349)

In [47]:
fit!(vlmm)

This is Ipopt version 3.12.10, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:       28

Total number of variables............................:        7
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0 

┌ Warning: Optimization unsuccesful; got UserLimit
└ @ VarLMM /Users/christophergerman/.julia/dev/VarLMM/src/mom_nlp.jl:43


──────────────────────────────────────────────────────────
      Estimate     Std. Error  Wald Statistic  Pr(>|Wald|)
──────────────────────────────────────────────────────────
β1  80.17           1.13473           4991.58       <1e-99
β2  15.1241         0.280195          2913.52       <1e-99
β3  -3.09795        0.439883            49.60       <1e-11
β4   0.908544       0.0519011          306.44       <1e-67
β5  -0.0969026      0.559873             0.03       0.8626
τ1  -0.180466   11167.2                  0.00       1.0000
τ2   1.07071     1163.14                 0.00       0.9993
τ3  -1.8218      1679.25                 0.00       0.9991
τ4   0.0733521    330.233                0.00       0.9998
──────────────────────────────────────────────────────────

In [48]:
fitweightedonly!(vlmm; weightedruns=2)

This is Ipopt version 3.12.10, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:       28

Total number of variables............................:        7
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0 

───────────────────────────────────────────────────────
      Estimate  Std. Error  Wald Statistic  Pr(>|Wald|)
───────────────────────────────────────────────────────
β1  78.9888      1.84882           1825.33       <1e-99
β2  15.1484      0.285537          2814.55       <1e-99
β3  -3.11466     0.455887            46.68       <1e-11
β4   0.965104    0.0862324          125.26       <1e-28
β5  -0.255645    0.57627              0.20       0.6573
τ1  -0.143219    0.997315             0.02       0.8858
τ2   1.07065     0.0620637          297.59       <1e-65
τ3  -1.82311     0.115662           248.45       <1e-55
τ4   0.0715455   0.0425644            2.83       0.0928
───────────────────────────────────────────────────────

In [63]:
using Random, LinearAlgebra, Distributions, DataFrames, StatsBase
@info "generate data"
Random.seed!(123)
# dimensions
m  = 500 # number of individuals
ns = rand(9:11, m) # numbers of observations per individual
p  = 5    # number of fixed effects, including intercept
q  = 2    # number of random effects, including intercept
l  = 4    # number of WS variance covariates, including intercept
obsvec = Vector{VarLmmObs{Float64}}(undef, m)
# true parameter values
βtrue = [80.0; 15.0; -2.5; 0.9; 0.75] #intercept, agegroup, #gender, #bmi, #meds
τtrue = [-1.0; 1.5; -1.5; 1.0] #intercept, agegroup, meds, bmi
Σγ    = Matrix(Diagonal([2.0; 1.0]))
δγω   = [0.1; 0.1; rand(q - 2) ./ 10]
σω    = [0.5]
Σγω   = [Σγ δγω; δγω' σω]
Lγω   = cholesky(Symmetric(Σγω), check = false).L
Lγ    = Lγω[1:q, 1:q]
lγω   = Lγω[q + 1, 1:q]
lω    = Lγω[q + 1, q + 1]
# generate data
γω = Vector{Float64}(undef, q + 1)
z  = similar(γω) # hold vector of iid std normal
df = DataFrame(id = String[], sbp = Float64[], age=Float64[], 
    gender=Int64[], bmi = Float64[], meds = Float64[])
bmis = zscore(Distributions.rand(Normal(25, 1.2), m))
for i in 1:m
    # first column intercept, remaining entries iid std normal
    X = Matrix{Float64}(undef, ns[i], p)
    X[:, 1] .= 1
    agegroup = Distributions.rand(1:3) #age
    gender = Distributions.rand(Bernoulli(0.5)) #gender
    meds = Distributions.rand(Bernoulli(0.2)) #meds
    @views fill!(X[:, 2], agegroup)
    @views fill!(X[:, 3], Int(gender))
    @views fill!(X[:, 4], bmis[i]) #bmi
    @views fill!(X[:, 5], meds)
    # first column intercept, remaining entries iid std normal
    Z = Matrix{Float64}(undef, ns[i], q)
    Z[:, 1] .= 1
    @views copyto!(Z[:, 2], X[:, 4]) #bmi 
    # first column intercept, remaining entries iid std normal
    W = Matrix{Float64}(undef, ns[i], l)
    W[:, 1] .= 1
    @views copyto!(W[:, 2], X[:, 2]) #agegroup
    @views copyto!(W[:, 3], X[:, 5]) #meds
    @views copyto!(W[:, 4], X[:, 4]) #bmi
    # generate random effects: γω = Lγω * z
    mul!(γω, Lγω, Distributions.rand!(Normal(), z))
    # generate y
    μy = X * βtrue + Z * γω[1:q]
    @views vy = exp.(W * τtrue .+ dot(γω[1:q], lγω) .+ γω[end])
    y = rand(MvNormal(μy, Diagonal(vy)))
        if i == 1
        @show vy
        @show μy
        @show y
    end
    id = fill(string(i), ns[i])
    tempdf = DataFrame([id y X[:, 2:p]])
    rename!(tempdf, [:id, :sbp, :age, :gender, :bmi, :meds])
    # form a VarLmmObs instance
    append!(df, tempdf)
end
rename!(df, [:id, :sbp, :age, :gender, :bmi, :meds])


vlmm = VarLmmModel(@formula(sbp ~ 1 + age + gender + bmi + meds), 
    @formula(sbp ~ 1 + bmi), 
    @formula(sbp ~ 1 + age + meds + bmi),
    :id, df);

ReVars = Vector{Float64}(undef, m)
WsVars = Vector{Float64}(undef, m)
RevarWsVarRatio = Vector{Float64}(undef, m)

for i in eachindex(vlmm.data)
    ob = vlmm.data[i]
    ReVars[i] = tr(transpose(ob.Zt) * Lγ * transpose(Lγ) * ob.Zt)
    WsVars[i] = sum(exp.(transpose(ob.Wt) * τtrue))
    RevarWsVarRatio[i] = ReVars[i] / WsVars[i]
end
mean(RevarWsVarRatio), minimum(RevarWsVarRatio), maximum(RevarWsVarRatio)

┌ Info: generate data
└ @ Main In[63]:2


vy = [29.454943493875714, 29.454943493875714, 29.454943493875714, 29.454943493875714, 29.454943493875714, 29.454943493875714, 29.454943493875714, 29.454943493875714, 29.4549434938757]
μy = [124.54364969500982, 124.54364969500982, 124.54364969500982, 124.54364969500982, 124.54364969500982, 124.54364969500982, 124.54364969500982, 124.54364969500982, 124.54364969500982]
y = [124.6326742539616, 119.60134146006015, 130.74399099763633, 136.7623626070695, 125.24683472139415, 117.30725729646784, 135.5351170662772, 132.15622809151768, 123.19559003795935]


(2.4277987507494685, 0.01719020250984096, 106.3583124753457)

In [64]:
VarLMM.fit!(vlmm)

This is Ipopt version 3.12.10, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:       28

Total number of variables............................:        7
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0 

┌ Warning: Ipopt finished with status Invalid_Number_Detected
└ @ Ipopt /Users/christophergerman/.julia/packages/Ipopt/ruIXY/src/MPB_wrapper.jl:178
┌ Warning: Optimization unsuccesful; got Error
└ @ VarLMM /Users/christophergerman/.julia/dev/VarLMM/src/mom_nlp.jl:43


──────────────────────────────────────────────────────
     Estimate  Std. Error  Wald Statistic  Pr(>|Wald|)
──────────────────────────────────────────────────────
β1  NaN               NaN             NaN          NaN
β2  NaN               NaN             NaN          NaN
β3  NaN               NaN             NaN          NaN
β4  NaN               NaN             NaN          NaN
β5  NaN               NaN             NaN          NaN
τ1    3.58871         NaN             NaN          NaN
τ2    0.0             NaN             NaN          NaN
τ3    0.0             NaN             NaN          NaN
τ4    0.0             NaN             NaN          NaN
──────────────────────────────────────────────────────

In [62]:
describe(df)

,variable,mean,min,median,max,nunique,nmissing,eltype
,Symbol,Union…,Any,Union…,Any,Union…,Nothing,DataType
1,id,,1,,99,500,,String
2,sbp,108.688,71.8918,108.342,168.271,,,Float64
3,age,1.98563,1.0,2.0,3.0,,,Float64
4,gender,0.50888,0,1.0,1,,,Int64
5,bmi,0.00841464,-2.85131,0.0592659,3.10662,,,Float64
6,meds,0.200559,0.0,0.0,1.0,,,Float64


In [65]:
init_ls!(vlmm)

VarLmmModel{Float64}(VarLmmObs{Float64}[VarLmmObs{Float64}([124.6326742539616, 119.60134146006015, 130.74399099763633, 136.7623626070695, 125.24683472139415, 117.30725729646784, 135.5351170662772, 132.15622809151768, 123.19559003795935], [1.0 1.0 … 1.0 1.0; 3.0 3.0 … 3.0 3.0; … ; 0.31558162843168314 0.31558162843168314 … 0.31558162843168314 0.31558162843168314; 0.0 0.0 … 0.0 0.0], [1.0 1.0 … 1.0 1.0; 0.31558162843168314 0.31558162843168314 … 0.31558162843168314 0.31558162843168314], [1.0 1.0 … 1.0 1.0; 3.0 3.0 … 3.0 3.0; 0.0 0.0 … 0.0 0.0; 0.31558162843168314 0.31558162843168314 … 0.31558162843168314 0.31558162843168314], [0.0, 5.0e-324, 2.1740810925e-314, 5.0e-324, 5.0e-324], [NaN, NaN, NaN, NaN], [NaN NaN; NaN NaN], [NaN NaN NaN NaN; NaN NaN NaN NaN; NaN NaN NaN NaN; NaN NaN NaN NaN], [NaN NaN NaN; NaN NaN NaN; NaN NaN NaN; NaN NaN NaN], [NaN NaN NaN; NaN NaN NaN; NaN NaN NaN], [-0.7518560173017477, -5.783188811203189, 5.359460726372987, 11.377832335806147, -0.13769554986919275, -8.0

In [66]:
vlmm.τ

4-element Array{Float64,1}:
 3.5887096104383414
 0.0               
 0.0               
 0.0               

In [67]:
vlmm.Lγ

2×2 Array{Float64,2}:
  2.39703      0.0       
 -1.42321e10  -2.02552e20

In [68]:
using Random, LinearAlgebra, Distributions, DataFrames, StatsBase
@info "generate data"
Random.seed!(123)
# dimensions
m  = 500 # number of individuals
ns = rand(9:11, m) # numbers of observations per individual
p  = 5    # number of fixed effects, including intercept
q  = 2    # number of random effects, including intercept
l  = 4    # number of WS variance covariates, including intercept
obsvec = Vector{VarLmmObs{Float64}}(undef, m)
# true parameter values
βtrue = [80.0; 15.0; -2.5; 0.9; 0.75] #intercept, agegroup, #gender, #bmi, #meds
τtrue = [-1.0; 1.5; -1.5; 1.0] #intercept, agegroup, meds, bmi
Σγ    = Matrix(Diagonal([2.0; 1.2]))
δγω   = [0.1; 0.1; rand(q - 2) ./ 10]
σω    = [0.5]
Σγω   = [Σγ δγω; δγω' σω]
Lγω   = cholesky(Symmetric(Σγω), check = false).L
Lγ    = Lγω[1:q, 1:q]
lγω   = Lγω[q + 1, 1:q]
lω    = Lγω[q + 1, q + 1]
# generate data
γω = Vector{Float64}(undef, q + 1)
z  = similar(γω) # hold vector of iid std normal
df = DataFrame(id = String[], sbp = Float64[], age=Float64[], 
    gender=Int64[], bmi = Float64[], meds = Float64[])
bmis = zscore(Distributions.rand(Normal(25, 1.2), m))
for i in 1:m
    # first column intercept, remaining entries iid std normal
    X = Matrix{Float64}(undef, ns[i], p)
    X[:, 1] .= 1
    agegroup = Distributions.rand(1:3) #age
    gender = Distributions.rand(Bernoulli(0.5)) #gender
    meds = Distributions.rand(Bernoulli(0.2)) #meds
    @views fill!(X[:, 2], agegroup)
    @views fill!(X[:, 3], Int(gender))
    @views fill!(X[:, 4], bmis[i]) #bmi
    @views fill!(X[:, 5], meds)
    # first column intercept, remaining entries iid std normal
    Z = Matrix{Float64}(undef, ns[i], q)
    Z[:, 1] .= 1
    @views copyto!(Z[:, 2], X[:, 4]) #bmi 
    # first column intercept, remaining entries iid std normal
    W = Matrix{Float64}(undef, ns[i], l)
    W[:, 1] .= 1
    @views copyto!(W[:, 2], X[:, 2]) #agegroup
    @views copyto!(W[:, 3], X[:, 5]) #meds
    @views copyto!(W[:, 4], X[:, 4]) #bmi
    # generate random effects: γω = Lγω * z
    mul!(γω, Lγω, Distributions.rand!(Normal(), z))
    # generate y
    μy = X * βtrue + Z * γω[1:q]
    @views vy = exp.(W * τtrue .+ dot(γω[1:q], lγω) .+ γω[end])
    y = rand(MvNormal(μy, Diagonal(vy)))
        if i == 1
        @show vy
        @show μy
        @show y
    end
    id = fill(string(i), ns[i])
    tempdf = DataFrame([id y X[:, 2:p]])
    rename!(tempdf, [:id, :sbp, :age, :gender, :bmi, :meds])
    # form a VarLmmObs instance
    append!(df, tempdf)
end
rename!(df, [:id, :sbp, :age, :gender, :bmi, :meds])


vlmm = VarLmmModel(@formula(sbp ~ 1 + age + gender + bmi + meds), 
    @formula(sbp ~ 1 + bmi), 
    @formula(sbp ~ 1 + age + meds + bmi),
    :id, df);

ReVars = Vector{Float64}(undef, m)
WsVars = Vector{Float64}(undef, m)
RevarWsVarRatio = Vector{Float64}(undef, m)

for i in eachindex(vlmm.data)
    ob = vlmm.data[i]
    ReVars[i] = tr(transpose(ob.Zt) * Lγ * transpose(Lγ) * ob.Zt)
    WsVars[i] = sum(exp.(transpose(ob.Wt) * τtrue))
    RevarWsVarRatio[i] = ReVars[i] / WsVars[i]
end
mean(RevarWsVarRatio), minimum(RevarWsVarRatio), maximum(RevarWsVarRatio)

┌ Info: generate data
└ @ Main In[68]:2


vy = [29.39138829150346, 29.39138829150346, 29.39138829150346, 29.39138829150346, 29.39138829150346, 29.39138829150346, 29.39138829150346, 29.39138829150346, 29.391388291503446]
μy = [124.54891144088906, 124.54891144088906, 124.54891144088906, 124.54891144088906, 124.54891144088906, 124.54891144088906, 124.54891144088906, 124.54891144088906, 124.54891144088906]
y = [124.63783990342117, 119.61193811724819, 130.74255986452843, 136.75443501982855, 125.2513374232051, 117.32033027341697, 135.52851421351312, 132.1532725371708, 123.20230692956552]


(2.648543436714589, 0.019972817485802218, 123.43023434311377)

In [69]:
init_ls!(vlmm)
vlmm.Lγ

2×2 Array{Float64,2}:
  2.43952     0.0       
 -6.92672e9  -4.79795e19